In [ ]:
import os
import sys
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as Function
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader

from model.dataset import SMPLRandomDataset
from model.SkinningNet import JointNet, WeightBindingNet, SkinningNet
from model.utils import SkinningLoss

from meshplot import plot
import igl
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:%s"%device)

In [ ]:
batchSize = 8
trainDataset = SMPLRandomDataset(complexity = "all", 
                                 gender = "mixed", 
                                 dataSize = 5000, 
                                 vertexOrderRandom = True, 
                                 noise = 0, 
                                 rotate = True, 
                                 scale = True,
                                 translate = True,
                                 centre = False)
trainIter = DataLoader(trainDataset, num_workers=0, batch_size=batchSize, shuffle=True, drop_last=True)
allTrainNUM = len(trainDataset)

In [ ]:
jointNet = JointNet()
jointNet.load_state_dict(torch.load(".\stateDict\jointNet_pretrain_rotation.pkl"))

weightNet = WeightBindingNet(jointNum = 24)
weightNet.load_state_dict(torch.load(".\stateDict\weightNet_pretrain_rotation.pkl"))

net = SkinningNet(jointNet, weightNet)
#net = SkinningNet()
net = net.to(device)

In [ ]:
lr = 0.00005
epochs = 100
opt = torch.optim.Adam(net.parameters(), lr=lr)
scheduler = CosineAnnealingLR(opt, epochs, eta_min=0.1*lr)
loss = SkinningLoss()

In [ ]:
beginTime = time.time()
net = net.train()

for epoch in range(epochs):
    epochBegin = time.time()
    trainLoss = 0.0
    completedNum = 0
    batchNum = 0
    trainAcc = 0.0
    for V, facesOneRingIdx, rigW, joints in trainIter:
        V = V.to(device).float()
        joints = joints.to(device).float()
        facesOneRingIdx = facesOneRingIdx.to(device).long()
        rigW = rigW.to(device).float()
        
        opt.zero_grad()
        
        preJ, preA = net(V, facesOneRingIdx)
        jl, sl, acc = loss(preJ, preA, joints, rigW.permute(0, 2, 1))
        l = jl+sl
        l.backward()
        opt.step()
        
        batchLoss = float(jl)
        batchAcc = float(acc)
        trainAcc += batchAcc
        trainLoss += batchLoss
        batchNum += 1
        completedNum += batchSize
        compltePer = completedNum/allTrainNUM
        leftTime = ((time.time() - epochBegin)/compltePer - (time.time() - epochBegin))/60
        sys.stdout.write('\r 训练%i / %i, 关节点损失: %f, 准确率：%0.2f%%, 当前进度: %0.2f%%, 预计剩余%d分钟'%(completedNum, allTrainNUM, batchLoss, batchAcc*100, compltePer * 100, leftTime))
        sys.stdout.flush()
        
    torch.save(net.state_dict(), ".\stateDict\epochs\skinningNet_%d.pkl"%epoch)
    print("\n")
    print("epoch%d, 关节点损失: %f, 准确率：%0.2f%%, 用时：%0.2f minuetes"%(epoch+1, trainLoss/batchNum, trainAcc/batchNum*100, (time.time() - beginTime) / 60 ))
    print("----------------------------------")
    scheduler.step()

In [ ]:
torch.save(net.state_dict(), ".\stateDict\skinningNet_fineture_rotation.pkl")

In [ ]:
batchSize = 8
testDataset = SMPLRandomDataset(complexity = "all", 
                                 gender = "mixed", 
                                 dataSize = 5000, 
                                 vertexOrderRandom = True, 
                                 noise = 0, 
                                 rotate = False, 
                                 scale = False,
                                 translate = False,
                                 centre = True)
testIter = DataLoader(testDataset, num_workers=0, batch_size=batchSize, shuffle=True, drop_last=True)
allTestNUM = len(testDataset)

In [ ]:
net = SkinningNet(transform = False)
net = net.to(device)
net.load_state_dict(torch.load(".\stateDict\skinningNet_fineture_rotation.pkl"))
loss = SkinningLoss()

In [ ]:
testLoss = 0.0
completedNum = 0
batchNum = 0
testAcc = 0.0
timeBegin = time.time()
net = net.eval()
for V, facesOneRingIdx, rigW, joints in testIter:
    V = V.to(device).float()
    joints = joints.to(device).float()
    facesOneRingIdx = facesOneRingIdx.to(device).long()
    rigW = rigW.to(device).float()
    with torch.no_grad():
        preJ, preA = net(V, facesOneRingIdx)
        jl, sl, acc =loss(preJ, preA, joints, rigW.permute(0, 2, 1))
        l = jl+sl

    batchLoss = float(jl)
    batchAcc = float(acc)
    testAcc += batchAcc
    testLoss += batchLoss
    batchNum += 1
    completedNum += batchSize
    compltePer = completedNum/allTestNUM
    leftTime = ((time.time() - timeBegin)/compltePer - (time.time() - timeBegin))/60
    sys.stdout.write('\r 测试%i / %i, 关节点损失: %f, 准确率：%0.2f%%, 当前进度: %0.2f%%, 预计剩余%d分钟'%(completedNum, allTestNUM, batchLoss, batchAcc*100, compltePer * 100, leftTime))
    sys.stdout.flush()
print("测试完成，关节点损失: %0f%%, 平均准确率：%0.2f%%"%(testLoss, testAcc))

In [ ]:
from LBS import LBS, getBetaMatrix
from SkeletonTransfer import skeletonTransfer, skeletonTransferWithVirtualJoints, AddVirtualJoints